In [1]:
import pandas as pd
import os
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.embeddings.base import Embeddings
import requests
from dotenv import load_dotenv
from typing import List
from tenacity import retry, stop_after_attempt, wait_random_exponential

In [2]:
data = pd.read_csv('KnowledgeBase.csv')
data.head()

,Топик,Подтопик,Поисковой запрос,Текст
0,Главное,Приложение Мой МТС,Как проверить последние пополнения и списания,1. Откройте Расходы\n\n\n2. По умолчанию отоб...
1,Главное,Приложение Мой МТС,Как заказать детализацию,"\nВ детализации есть полные данные о том, как ..."
2,Главное,Приложение Мой МТС,Как оставаться на связи при минусе,Есть несколько способов пользоваться номером п...
3,Главное,Приложение Мой МТС,Как проверить баланс и пакеты по тарифу,"На главном экране, который открывается при вхо..."
4,Главное,Приложение Мой МТС,"Что делать, если остатки пакетов не показываются","Вы не увидите остатки интернета, минут или СМС..."


In [3]:
data.columns = ['topic', 'subtopic', 'query', 'text']
data.head()

,topic,subtopic,query,text
0,Главное,Приложение Мой МТС,Как проверить последние пополнения и списания,1. Откройте Расходы\n\n\n2. По умолчанию отоб...
1,Главное,Приложение Мой МТС,Как заказать детализацию,"\nВ детализации есть полные данные о том, как ..."
2,Главное,Приложение Мой МТС,Как оставаться на связи при минусе,Есть несколько способов пользоваться номером п...
3,Главное,Приложение Мой МТС,Как проверить баланс и пакеты по тарифу,"На главном экране, который открывается при вхо..."
4,Главное,Приложение Мой МТС,"Что делать, если остатки пакетов не показываются","Вы не увидите остатки интернета, минут или СМС..."


In [4]:
data = data['query']
data.head()

0       Как проверить последние пополнения и списания
1                            Как заказать детализацию
2                  Как оставаться на связи при минусе
3             Как проверить баланс и пакеты по тарифу
4    Что делать, если остатки пакетов не показываются
Name: query, dtype: object

In [ ]:
class MTS_Embeddings(Embeddings):
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.api_url = "https://api.gpt.mws.ru/v1/embeddings"
        
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self._get_embedding(text) for text in texts]
    
    def embed_query(self, text: str) -> List[float]:
        return self._get_embedding(text)
        
    @retry(wait=wait_random_exponential(min=20, max=50), stop=stop_after_attempt(200))
    def _get_embedding(self, text: str) -> List[float]:
        headers = {'Authorization': f'Bearer {self.api_key}'}
        data = {
                "model": "bge-m3",
                "input": text
            }
        try:
            response = requests.post(self.api_url, json=data, headers=headers)
            return response.json()['data'][0]['embedding']    
        except Exception as e:
            print(f'{e}: embedding error')
            return e
        
    

In [6]:
def create_and_save_faiss_db(questions_series: pd.Series, api_key: str, save_path: str):
    """
    Creates a FAISS vector store from the questions in the pandas Series and saves it locally.
    
    Args:
        questions_series (pd.Series): A pandas Series containing the questions.
        api_key (str): The API key for the embedding model.
        save_path (str): The local directory path to save the FAISS index.
    """
    embedder = MTS_Embeddings(api_key)
    documents = [Document(page_content=question) for question in questions_series]
    vector_store = FAISS.from_documents(documents, embedder)
    vector_store.save_local(save_path)

In [ ]:
def get_top_3_similar_questions_from_db(save_path: str, api_key: str, input_question: str) -> List[str]:
    """
    Loads the saved FAISS vector store and retrieves the top 3 most similar questions to the input question.
    
    Args:
        save_path (str): The local directory path where the FAISS index is saved.
        api_key (str): The API key for the embedding model.
        input_question (str): The input question to find similar questions for.
    
    Returns:
        List[str]: A list of the top 3 most similar questions.
    """
    embedder = MTS_Embeddings(api_key)
    vector_store = FAISS.load_local(save_path, embeddings=embedder, allow_dangerous_deserialization=True)
    similar_docs = vector_store.similarity_search(input_question, k=3)
    top_questions = [doc.page_content for doc in similar_docs]
    """_summary_

    Returns:
        _type_: _description_
    """    return top_questions

In [8]:
load_dotenv('.env.private', override=True)
api_key = os.getenv('API_KEY')

In [11]:
save_path = "faiss_index"
# create_and_save_faiss_db(data, api_key, save_path)

In [12]:
input_question = "Чем КАСКО отличается от ОСАГО ?"
top_3_questions = get_top_3_similar_questions_from_db(save_path, api_key, input_question)
print(top_3_questions)

['В чем различие между ОСАГО и КАСКО?', 'Для чего нужно КАСКО?', 'Что покрывает КАСКО']
